The function `nr_pusch_channel_estimation` is called within the `nr_rx_pusch_tp` function to perform channel estimation for each antenna and each layer of the received PUSCH (Physical Uplink Shared Channel) signal. Let's break down its usage and the overall context:

### Context in `nr_rx_pusch_tp`

1. **Function Definition**:
   ```c
   int nr_rx_pusch_tp(PHY_VARS_gNB *gNB,
                      uint8_t ulsch_id,
                      uint32_t frame,
                      uint8_t slot,
                      unsigned char harq_pid)
   ```

2. **Initialization**:
   - The function initializes various parameters, including `frame_parms`, `rel15_ul`, and `pusch_vars`.
   - It determines the start subcarrier for the BWP (Bandwidth Part) and logs some information.

3. **Channel Estimation Loop**:
   - The main part of the function is a loop over the symbols in the PUSCH transmission:
     ```c
     for(uint8_t symbol = rel15_ul->start_symbol_index; symbol < (rel15_ul->start_symbol_index + rel15_ul->nr_of_symbols); symbol++) {
       uint8_t dmrs_symbol_flag = (rel15_ul->ul_dmrs_symb_pos >> symbol) & 0x01;
       if (dmrs_symbol_flag == 1) {
         if (pusch_vars->dmrs_symbol == INVALID_VALUE)
           pusch_vars->dmrs_symbol = symbol;
         for (int nl = 0; nl < rel15_ul->nrOfLayers; nl++) {
           uint32_t nvar_tmp = 0;
           nr_pusch_channel_estimation(gNB,
                                       slot,
                                       nl,
                                       get_dmrs_port(nl, rel15_ul->dmrs_ports),
                                       symbol,
                                       ulsch_id,
                                       bwp_start_subcarrier,
                                       rel15_ul,
                                       &max_ch,
                                       &nvar_tmp);
           nvar += nvar_tmp;
         }
       }
     }
     ```

### Usage of `nr_pusch_channel_estimation`

The `nr_pusch_channel_estimation` function is called with the following parameters:
- **gNB**: A pointer to the gNB (gNodeB) structure, which contains all the necessary context and parameters for the NR (New Radio) base station.
- **slot**: The slot number within the frame.
- **nl**: The current layer index.
- **get_dmrs_port(nl, rel15_ul->dmrs_ports)**: The DMRS (Demodulation Reference Signal) port number for the current layer.
- **symbol**: The current symbol index within the slot.
- **ulsch_id**: The identifier for the ULSCH (Uplink Shared Channel) process.
- **bwp_start_subcarrier**: The starting subcarrier index for the BWP.
- **rel15_ul**: A pointer to the PUSCH PDU (Protocol Data Unit) structure, containing all the PUSCH-specific parameters.
- **&max_ch**: A pointer to an integer where the maximum channel value will be stored.
- **&nvar_tmp**: A pointer to a variable where the noise variance for the current estimation will be stored.

### Breakdown of `nr_pusch_channel_estimation` Role

1. **Channel Estimation**:
   - The function performs channel estimation for the given symbol and layer using the received DMRS signals.
   - It estimates the channel response between the transmitter (UE) and receiver (gNB) for each antenna.

2. **Update `max_ch`**:
   - During the channel estimation process, the function updates `max_ch` to reflect the maximum channel gain observed. This is useful for normalizing the channel estimates and ensuring the dynamic range is handled appropriately.

3. **Noise Variance Calculation (`nvar_tmp`)**:
   - The function calculates the noise variance (`nvar_tmp`) for the current symbol and layer. This provides a measure of the noise power in the received signal, which is important for SNR (Signal-to-Noise Ratio) estimation and subsequent decoding steps.

4. **Accumulation of Noise Variance**:
   - After each call to `nr_pusch_channel_estimation`, the accumulated noise variance (`nvar_tmp`) is added to a running total (`nvar`).
   - This total is later averaged to obtain the overall noise variance for the entire PUSCH transmission.

### Summary of the Process

1. **Loop over Symbols**:
   - For each symbol in the PUSCH transmission, check if it contains DMRS.
   - If it does, proceed with channel estimation.

2. **Loop over Layers**:
   - For each layer, call `nr_pusch_channel_estimation` to perform the actual channel estimation and noise variance calculation.

3. **Update Variables**:
   - Update `max_ch` with the maximum channel gain observed.
   - Accumulate `nvar_tmp` into `nvar`.

4. **Averaging Noise Variance**:
   - After the loops, average the accumulated noise variance (`nvar`) over the number of symbols, layers, and antennas.

By using `nr_pusch_channel_estimation` in this way, the function `nr_rx_pusch_tp` ensures that channel estimates are accurately computed for each symbol and layer, and that noise variance is properly accounted for, which are crucial for reliable decoding of the PUSCH.